In [55]:
import pandas as pd
import numpy as np

### Preprocessing

In [56]:
data = pd.read_csv('twitter_data/tweets_extended_fields.csv')
data

/var/folders/pv/j2wxhdx50y5fbwqrc2yxqwf00000gn/T/ipykernel_57494/2236879572.py:1: DtypeWarning: Columns (14,29,34) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('twitter_data/tweets_extended_fields.csv')


,possibly_sensitive,conversation_id,in_reply_to_user_id,author_id,Tweets,public_metrics,source,lang,referenced_tweets,created_at,edit_history_tweet_ids,entities,id,attachments,geo,author,Branche,Branche-6,Sektor,B2B,B2C,likes,retweets,replies,quotes,quote_ids,reply_ids,retweet_ids,media_keys,poll_ids,mentions,annotations,urls,hashtags,cashtags,entity_types,entity_names,mentioned_usernames,mentioned_hashtags
0,False,1005377380997828608,1.843782e+09,54502344,"@castleburyexec Hi Andy, we want to help you h...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",Unymira Connect,en,"[{'type': 'replied_to', 'id': '100537738099782...",2018-06-09T16:27:21.000Z,['1005486517891067906'],"{'annotations': [{'start': 19, 'end': 22, 'pro...",1005486517891067906,{'media_keys': {}},{},MercedesBenz,automobil,Automobil,2.0,True,True,0,0,0,0,[],['1005377380997828608'],[],{},NaN,{},"[{'start': 19, 'end': 22, 'probability': 0.692...",{},{},NaN,['Person'],['Andy'],[],[]
1,False,1005253183902543872,7.746454e+17,54502344,@neo_ysl GLG 2020. It is yet to be confirm whe...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Unymira Connect,en,"[{'type': 'replied_to', 'id': '100527567064836...",2018-06-09T16:14:04.000Z,['1005483173244030976'],"{'annotations': [{'start': 9, 'end': 16, 'prob...",1005483173244030976,{'media_keys': {}},{},MercedesBenz,automobil,Automobil,2.0,True,True,1,0,0,0,[],['1005275670648369153'],[],{},NaN,"[{'start': 0, 'end': 8, 'username': 'neo_ysl',...","[{'start': 9, 'end': 16, 'probability': 0.7305...",{},{},NaN,['Other'],['GLG 2020'],['neo_ysl'],[]
2,False,988516398077886464,2.234039e+07,54502344,@aWmeown Thanks for your contact. We're sorry ...,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",Unymira Connect,en,"[{'type': 'replied_to', 'id': '100526911657861...",2018-06-09T16:04:27.000Z,['1005480753512185856'],"{'annotations': {}, 'mentions': {}, 'urls': {}...",1005480753512185856,{'media_keys': {}},{},MercedesBenz,automobil,Automobil,2.0,True,True,0,1,0,0,[],['1005269116578619393'],[],{},NaN,{},{},{},{},NaN,[],[],[],[]
3,False,1005201522328649728,2.755694e+08,50883209,@otaubman We want to get to the bottom of this...,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",Customer online social care.,en,"[{'type': 'replied_to', 'id': '100520152232864...",2018-06-09T16:01:12.000Z,['1005479935526555657'],"{'annotations': {}, 'mentions': [{'start': 0, ...",1005479935526555657,{'media_keys': {}},{},PUMA,mode,Textil,2.0,True,True,0,0,1,0,[],['1005201522328649728'],[],{},NaN,"[{'start': 0, 'end': 9, 'username': 'otaubman'...",{},"[{'start': 67, 'end': 90, 'url': 'https://t.co...",{},NaN,[],[],['otaubman'],[]
4,False,1005479690621026306,NaN,54502344,The Berlin State of Mind.\n\nEnjoy the true de...,"{'retweet_count': 95, 'reply_count': 7, 'like_...",erased6981162,en,{},2018-06-09T16:00:13.000Z,['1005479690621026306'],"{'annotations': [{'start': 4, 'end': 15, 'prob...",1005479690621026306,{'media_keys': ['3_1005479688762994691']},{},MercedesBenz,automobil,Automobil,2.0,True,True,829,95,7,3,[],[],[],['3_1005479688762994691'],NaN,{},"[{'start': 4, 'end': 15, 'probability': 0.3684...","[{'start': 72, 'end': 95, 'url': 'https://t.co...","[{'start': 119, 'end': 131, 'tag': 'MBsocialca...",NaN,"['Organization', 'Other', 'Person', 'Organizat...","['Berlin State', 'Mind', 'Constantin Schiller'...",[],"['MBsocialcar', 'MercedesBenz']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642892,False,998980817811582976,4.185260e+08,1545994664,@CianHogan2 Thank you for your interest in BMW...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Sprinklr,en,"[{'type': 'replied_to', 'id': '998980817811582...",2018-05-22T21:42:26.000Z,['999042828746575875'],"{'mentions': [{'start': 0, 'end': 11, 'usernam...",999042828746575875,"{'media_keys': {}, 'poll_ids': {}}",{},BMW,automobil,Automobil,2.0,True,True,0,0,0,0,[],['998980817811582976'],[],{},{},

In [57]:
#format retweets data
retweet_ids = pd.read_csv('twitter_data/source_tweet_additional_text.csv')
retweet_ids['tweet_id'] = retweet_ids['tweet_id'].apply(lambda x: format(float(x), '.0f'))
retweet_ids['tweet_id'] = retweet_ids['tweet_id'].astype('int64')
retweet_ids = retweet_ids.rename(columns={"tweet_id": "id"})
retweet_ids = retweet_ids.drop(columns=['Unnamed: 0'])

In [58]:
data = data.merge(retweet_ids, how="left", on="id")

In [59]:
data['full_tweet'] = np.where(data['sourcetweet_text'].astype(str).str.len()< 5, data['Tweets'], data['sourcetweet_text'])
data = data.rename(columns={"Tweets": "tweets_original","full_tweet": "Tweets"})

In [60]:
data['year'] = pd.to_datetime(data['created_at']).dt.year
data['more_information_provided'] = (~data['urls'].isna())&~(data['urls']=="{}")

In [61]:
data = data.drop(columns=['public_metrics','author_id','attachments','entities','referenced_tweets','edit_history_tweet_ids','conversation_id','attachments','geo','hashtags','annotations','mentions'])

In [62]:
#remove other languages (23881 posts were non English and non German)
data = data[(data['lang'] == 'en') | (data['lang']=='de')]
data

,possibly_sensitive,in_reply_to_user_id,tweets_original,source,lang,created_at,id,author,Branche,Branche-6,Sektor,B2B,B2C,likes,retweets,replies,quotes,quote_ids,reply_ids,retweet_ids,media_keys,poll_ids,urls,cashtags,entity_types,entity_names,mentioned_usernames,mentioned_hashtags,sourcetweet_text,Tweets,year,more_information_provided
0,False,1.843782e+09,"@castleburyexec Hi Andy, we want to help you h...",Unymira Connect,en,2018-06-09T16:27:21.000Z,1005486517891067906,MercedesBenz,automobil,Automobil,2.0,True,True,0,0,0,0,[],['1005377380997828608'],[],{},NaN,{},NaN,['Person'],['Andy'],[],[],NaN,"@castleburyexec Hi Andy, we want to help you h...",2018,False
1,False,7.746454e+17,@neo_ysl GLG 2020. It is yet to be confirm whe...,Unymira Connect,en,2018-06-09T16:14:04.000Z,1005483173244030976,MercedesBenz,automobil,Automobil,2.0,True,True,1,0,0,0,[],['1005275670648369153'],[],{},NaN,{},NaN,['Other'],['GLG 2020'],['neo_ysl'],[],NaN,@neo_ysl GLG 2020. It is yet to be confirm whe...,2018,False
2,False,2.234039e+07,@aWmeown Thanks for your contact. We're sorry ...,Unymira Connect,en,2018-06-09T16:04:27.000Z,1005480753512185856,MercedesBenz,automobil,Automobil,2.0,True,True,0,1,0,0,[],['1005269116578619393'],[],{},NaN,{},NaN,[],[],[],[],NaN,@aWmeown Thanks for your contact. We're sorry ...,2018,False
3,False,2.755694e+08,@otaubman We want to get to the bottom of this...,Customer online social care.,en,2018-06-09T16:01:12.000Z,1005479935526555657,PUMA,mode,Textil,2.0,True,True,0,0,1,0,[],['1005201522328649728'],[],{},NaN,"[{'start': 67, 'end': 90, 'url': 'https://t.co...",NaN,[],[],['otaubman'],[],NaN,@otaubman We want to get to the bottom of this...,2018,True
4,False,NaN,The Berlin State of Mind.\n\nEnjoy the true de...,erased6981162,en,2018-06-09T16:00:13.000Z,1005479690621026306,MercedesBenz,automobil,Automobil,2.0,True,True,829,95,7,3,[],[],[],['3_1005479688762994691'],NaN,"[{'start': 72, 'end': 95, 'url': 'https://t.co...",NaN,"['Organization', 'Other', 'Person', 'Organizat...","['Berlin State', 'Mind', 'Constantin Schiller'...",[],"['MBsocialcar', 'MercedesBenz']",NaN,The Berlin State of Mind.\n\nEnjoy the true de...,2018,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642892,False,4.185260e+08,@CianHogan2 Thank you for your interest in BMW...,Sprinklr,en,2018-05-22T21:42:26.000Z,999042828746575875,BMW,automobil,Automobil,2.0,True,True,0,0,0,0,[],['998980817811582976'],[],{},{},"[{'start': 211, 'end': 234, 'url': 'https://t....",NaN,"['Other', 'Other']","['BMW Connected', 'BMW Connected']",['CianHogan2'],[],NaN,@CianHogan2 Thank you for your interest in BMW...,2018,True
642893,False,NaN,Are your tires properly inflated? Now is a goo...,Emplifi,en,2018-05-22T19:30:38.000Z,999009662685986816,continentaltire,automobil,Automobil,2.0,True,False,8,3,0,0,[],[],[],['3_999009661046083584'],{},"[{'start': 87, 'end': 110, 'url': 'https://t.c...",NaN,['Other'],['NationalTireSafetyWeek'],[],['NationalTireSafetyWeek'],NaN,Are your tires properly inflated? Now is a goo...,2018,True
642894,False,NaN,In order to sustain our planet's finite resour...,Twitter Ads,en,2018-05-22T18:47:00.000Z,998998679565451265,covestro,chemie,Technik,2.0,True,False,19,4,0,2,[],[],[],['13_998939017172455429'],{},"[{'start': 235, 'end': 258, 'url': 'https://t....",NaN,['Other'],['UTECHEurope'],[],"['InnovationForSustainableGrowth', 'UTECHEurope']",NaN,In order to sustain our planet's finite resour...,2018,True
642895,False,NaN,This year's #Indy500 pole sitter @edcarpenter2...,Emplifi,en,2018-05-22T17:32:53.000Z,998980027197800449,continentaltire,automobil,Automobil,2.0,True,False,3,1,0,0,[],[],[],['3_998980024995737600'],{},"[{'start': 203, 'end': 226, 'url': 'https://t....",NaN,['Other'],['Indy500'],"['edcarpenter20', 'DWRshow', 'IMS']",['Indy500'],NaN,This year's #Indy500 pole sitter @edcarpenter2...,2018,True


In [63]:
#removed all replies
data = data[data["in_reply_to_user_id"].isna()]
data = data.drop(columns=['in_reply_to_user_id','reply_ids'])

In [64]:
data

,possibly_sensitive,tweets_original,source,lang,created_at,id,author,Branche,Branche-6,Sektor,B2B,B2C,likes,retweets,replies,quotes,quote_ids,retweet_ids,media_keys,poll_ids,urls,cashtags,entity_types,entity_names,mentioned_usernames,mentioned_hashtags,sourcetweet_text,Tweets,year,more_information_provided
4,False,The Berlin State of Mind.\n\nEnjoy the true de...,erased6981162,en,2018-06-09T16:00:13.000Z,1005479690621026306,MercedesBenz,automobil,Automobil,2.0,True,True,829,95,7,3,[],[],['3_1005479688762994691'],NaN,"[{'start': 72, 'end': 95, 'url': 'https://t.co...",NaN,"['Organization', 'Other', 'Person', 'Organizat...","['Berlin State', 'Mind', 'Constantin Schiller'...",[],"['MBsocialcar', 'MercedesBenz']",NaN,The Berlin State of Mind.\n\nEnjoy the true de...,2018,True
8,False,It's officially #SportscarTogether day! How ar...,Twitter Web Client,en,2018-06-09T15:56:29.000Z,1005478751809961986,Porsche,automobil,Automobil,2.0,True,True,257,46,8,3,[],[],['7_1005478556346941443'],NaN,"[{'start': 81, 'end': 104, 'url': 'https://t.c...",NaN,"['Other', 'Other']","['SportscarTogether', 'Porsche70Years']",[],"['SportscarTogether', 'Porsche70Years']",NaN,It's officially #SportscarTogether day! How ar...,2018,True
25,False,With #Germany experiencing usually high temper...,Percolate,en,2018-06-09T13:22:03.000Z,1005439886764044290,MunichRe,versicherung,Finance/Insurance,3.0,True,False,13,4,1,0,[],[],{},NaN,"[{'start': 249, 'end': 272, 'url': 'https://t....",NaN,"['Place', 'Other', 'Person', 'Other']","['Germany', 'ExtremeWeather', 'Eberhard Faust'...",[],"['Germany', 'ExtremeWeather', 'climatechange',...",NaN,With #Germany experiencing usually high temper...,2018,True
26,False,The @FIAFormulaE comes to Zurich! Learn five t...,erased6981162,en,2018-06-09T13:04:15.000Z,1005435405473406976,MercedesBenz,automobil,Automobil,2.0,True,True,425,73,5,3,[],[],['13_1005434382600065026'],NaN,"[{'start': 244, 'end': 267, 'url': 'https://t....",NaN,"['Place', 'Other', 'Other', 'Other', 'Other', ...","['Zurich', 'Mercedes EQ', 'Allianz E-\n Villag...",['FIAFormulaE'],"['SwitchToEQ', 'EQ', 'ABBFormulaE', 'ZurichEpr...",NaN,The @FIAFormulaE comes to Zurich! Learn five t...,2018,True
32,False,"RT @MercedesAMGF1: Friday, that’s a wrap... 🎬 ...",Twitter Web Client,en,2018-06-09T08:28:41.000Z,1005366057174032385,MercedesBenz,automobil,Automobil,2.0,True,True,0,76,0,0,[],['1005201393806970881'],{},NaN,{},NaN,"['Person', 'Person', 'Other']","['Valtteri', 'Lewis', 'CanadianGP']",['MercedesAMGF1'],['CanadianGP'],NaN,"RT @MercedesAMGF1: Friday, that’s a wrap... 🎬 ...",2018,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642888,False,It's still not too late to meet the Brenntag B...,Twitter Web Client,en,2018-05-23T07:30:16.000Z,999190762117107712,Brenntag,chemie,Technik,2.0,True,False,5,1,0,0,[],[],['3_999190729376153600'],{},"[{'start': 172, 'end': 195, 'url': 'https://t....",NaN,"['Other', 'Other', 'Place', 'Place', 'Other', ...","['Brenntag Brazil Cosmetics Team', 'FCE Cosmet...",[],"['fcecosmetique', 'BrenntagBrazil']",NaN,It's still not too late to meet the Brenntag B...,2018,True
642889,False,RT @adidasrunning: There is a new dawn for run...,Twitter Ads,en,2018-05-22T22:20:56.000Z,999052517123108864,adidas,mode,Textil,2.0,True,True,0,280,0,0,[],['999038802822807552'],{},{},{},NaN,['Other'],['SOLARBOOST'],['adidasrunning'],['SOLARBOOST'],There is a new dawn for running. See how #SOLA...,There is a new dawn for running. See how #SOLA...,2018,False
642893,False,Are your tires properly inflated? Now is a goo...,Emplifi,en,2018-05-22T19:30:38.000Z,999009662685986816,continentaltire,automobil,Automobil,2.0,True,False,8,3,0,0,[],[],['3_999009661046083584'],{},"[{'start': 87, 'end': 110, 'url': 'https://t.c...",NaN,['Other'],['NationalTireSafetyWeek'],[],['NationalTireSafetyWeek'],NaN,Are your tires properly inflated? Now is a goo...,2018,True
642894,False,In order to sustain our plan

In [65]:
#data.to_csv('tweets_080123.csv', header=True, index=False, columns=list(data.axes[1]))

In [66]:
#sheet: "sustainab|sustain|climate|green|environment|ecofriendly|eco-friendly|solar|co2|biodegra|renewable|nature|natural |decarbon|recycl|pollut|waste|ecolog|emission|nachhaltig|klima|umwelt|ökolog|erneuerbar|global warming|wind energy|energy efficient"

word_list_en = ['sustainab','climate','green','environment','ecofriendly','eco-friendly','biodegra','renewable','netzero','greenbuild','decarbon','recycl', 'pollut','zero waste','wind energy','co2 neutral','co2-neutral','solar', 'emissions','global warming','fair trade']
word_list_de = ['nachhaltig', 'klima','umwelt','ökolog','erneuerbar','bio ','natur ']
word_list = word_list_de + word_list_en

word_list_en_recall = word_list_en + ['co2']
word_list_de_recall = word_list_de + ['biologisch']
word_list_high_recall = word_list_de + word_list_en

word_list_en_precision = [ elem for elem in word_list_en if not elem in ['green', 'climate', 'environment']]  
word_list_de_precision = [ elem for elem in word_list_de if not elem in ['klima','natur ']]  
word_list_high_precision = word_list_de + word_list_en

In [67]:
all = data.shape[0]
for word in word_list:
    print(word, round(data.loc[data['Tweets'].str.contains(word, na=False, case=False,regex=True)].shape[0]*100.0/all,2), "%")

nachhaltig 0.24 %
klima 0.22 %
umwelt 0.07 %
ökolog 0.02 %
erneuerbar 0.09 %
bio  0.03 %
natur  0.01 %
sustainab 2.56 %
climate 1.07 %
green 1.25 %
environment 0.79 %
ecofriendly 0.01 %
eco-friendly 0.03 %
biodegra 0.02 %
renewable 0.6 %
netzero 0.04 %
greenbuild 0.04 %
decarbon 0.2 %
recycl 0.2 %
pollut 0.07 %
zero waste 0.01 %
wind energy 0.03 %
co2 neutral 0.0 %
co2-neutral 0.04 %
solar 0.76 %
emissions 0.56 %
global warming 0.03 %
fair trade 0.0 %


In [68]:
#replace word_list with word_list_high_recall or word_list_high_precision alternatively for optimizing recall or precision, respectively
regex_query = ('|').join(word_list)
data['green'] = data['Tweets'].str.contains(regex_query, na=False, case=False,regex=True)
green = data.loc[data.green]

In [69]:
pd.set_option('display.max_columns', None)
green

,possibly_sensitive,tweets_original,source,lang,created_at,id,author,Branche,Branche-6,Sektor,B2B,B2C,likes,retweets,replies,quotes,quote_ids,retweet_ids,media_keys,poll_ids,urls,cashtags,entity_types,entity_names,mentioned_usernames,mentioned_hashtags,sourcetweet_text,Tweets,year,more_information_provided,green
25,False,With #Germany experiencing usually high temper...,Percolate,en,2018-06-09T13:22:03.000Z,1005439886764044290,MunichRe,versicherung,Finance/Insurance,3.0,True,False,13,4,1,0,[],[],{},NaN,"[{'start': 249, 'end': 272, 'url': 'https://t....",NaN,"['Place', 'Other', 'Person', 'Other']","['Germany', 'ExtremeWeather', 'Eberhard Faust'...",[],"['Germany', 'ExtremeWeather', 'climatechange',...",NaN,With #Germany experiencing usually high temper...,2018,True,True
88,False,Every minute one garbage truck of plastic is d...,Sprinklr Publishing,en,2018-06-08T10:17:19.000Z,1005031008415223808,Henkel,chemie,Technik,2.0,True,True,9,3,1,0,[],[],['16_1005030984193118208'],NaN,"[{'start': 277, 'end': 300, 'url': 'https://t....",NaN,['Place'],['Henkel'],[],['WorldOceansDay'],NaN,Every minute one garbage truck of plastic is d...,2018,True,True
95,False,"Can you imagine a world with fewer cars, no tr...",Percolate,en,2018-06-08T08:31:06.000Z,1005004276576354304,MunichRe,versicherung,Finance/Insurance,3.0,True,False,4,4,0,0,[],[],['3_1005004274672197632'],NaN,"[{'start': 249, 'end': 272, 'url': 'https://t....",NaN,[],[],"['Getaround', 'Maureeninno']",['startup'],NaN,"Can you imagine a world with fewer cars, no tr...",2018,True,True
115,False,RT @TerliWetter: #Klimawandel #Klimakrise ist ...,Twitter Web Client,de,2018-06-07T13:49:30.000Z,1004722015692050433,MunichRe,versicherung,Finance/Insurance,3.0,True,False,0,4,0,0,[],['1004691958667907072'],{},NaN,{},NaN,[],[],"['TerliWetter', 'MunichRe']","['Klimawandel', 'Klimakrise']",NaN,RT @TerliWetter: #Klimawandel #Klimakrise ist ...,2018,False,True
140,False,Driving #sustainability: Henkel Colombia prese...,Sprinklr Publishing,en,2018-06-07T11:17:31.000Z,1004683767934603264,Henkel,chemie,Technik,2.0,True,True,3,2,0,0,[],[],['3_1004683766072270850'],NaN,"[{'start': 279, 'end': 302, 'url': 'https://t....",NaN,"['Person', 'Other']","['Henkel Colombia', 'RSPOLatAm2018']",['Solidaridadnetw'],"['sustainability', 'RSPOLatAm2018', 'palmoil']",NaN,Driving #sustainability: Henkel Colombia prese...,2018,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642879,False,"At Allianz, #diversity plays an important role...",Twitter Web Client,en,2018-05-23T10:01:22.000Z,999228786485886976,Allianz,versicherung,Finance/Insurance,3.0,True,True,11,3,0,0,['999227261860630528'],[],{},{},"[{'start': 179, 'end': 202, 'url': 'https://t....",NaN,['Place'],['Allianz'],[],"['diversity', 'workplace']",NaN,"At Allianz, #diversity plays an important role...",2018,True,True
642883,False,RT @UNEnvironment: Humankind represents just 0...,Twitter Web Client,en,2018-05-23T08:11:05.000Z,999201035448090630,Allianz,versicherung,Finance/Insurance,3.0,True,True,0,311,0,0,[],['999198280276938752'],{},{},"[{'start': 113, 'end': 136, 'url': 'https://t....",NaN,[],[],[],[],NaN,RT @UNEnvironment: Humankind represents just 0...,2018,True,True
642884,False,As a world-leading supplier of high-tech #poly...,Twitter Ads,en,2018-05-23T07:58:00.000Z,999197741057265664,covestro,chemie,Technik,2.0,True,False,12,5,0,0,[],[],['13_998939017759600640'],{},"[{'start': 256, 'end': 279, 'url': 'https://t....",NaN,"['Other', 'Other']","['SDGs', 'UTECHEurope']",['Utech_PU'],"['polymer', 'SDGs', 'InnovationForSustainableG...",NaN,As a world-leading supplier of high-tech #poly...,2018,True,True
642889,False,RT @adidasrunning: There is a new dawn for run...,Twitter Ads,en,2018-05-22T22:20:56.000Z,999052517123108864,adidas,mode,Textil,2.0,True,True,0,280,0,0,[],['999038802822807552'],{},{},{},NaN,['Other'],['SOLARBOOST'],['adidasrunning'],['SOLARBOOST'],There is a new dawn

In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306131 entries, 4 to 642895
Data columns (total 31 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   possibly_sensitive         306131 non-null  bool   
 1   tweets_original            306131 non-null  object 
 2   source                     305668 non-null  object 
 3   lang                       306131 non-null  object 
 4   created_at                 306131 non-null  object 
 5   id                         306131 non-null  int64  
 6   author                     306131 non-null  object 
 7   Branche                    306131 non-null  object 
 8   Branche-6                  306131 non-null  object 
 9   Sektor                     306131 non-null  float64
 10  B2B                        306131 non-null  bool   
 11  B2C                        306131 non-null  bool   
 12  likes                      306131 non-null  int64  
 13  retweets                   30

In [71]:
data.to_csv('twitter_data/tweets_processed.csv', header=True, index=False, columns=list(data.axes[1]), lineterminator='\r\n')
green.to_csv('twitter_data/green_tweets_processed.csv', header=True, index=False, columns=list(green.axes[1]), lineterminator='\r\n')